Install Transformers Library

In [3]:
!pip install transformers


Import Required Modules

In [5]:
from transformers import pipeline, set_seed


In [6]:
generator = pipeline("text-generation", model="gpt2")
set_seed(42)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


Initialize Chat History

In [7]:
chat_history = ""


Simple Chat Loop

In [15]:
chat_history = ""

print("🤖 Chatbot: Hello! I’m a simple chatbot. Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("🤖 Chatbot: Goodbye!")
        break

    # Append user input to history
    chat_history += f"You: {user_input}\nChatbot:"

    # Generate response using GPT-2
    response = generator(chat_history, max_length=200, num_return_sequences=1, pad_token_id=50256)

    # Extract the chatbot's reply
    full_reply = response[0]["generated_text"]
    bot_reply = full_reply[len(chat_history):].split("\n")[0]

    print("🤖 Chatbot:", bot_reply.strip())

    # Update history
    chat_history += f" {bot_reply.strip()}\n"


🤖 Chatbot: Hello! I’m a simple chatbot. Type 'exit' to stop.

You: Can I call you with name John?


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


🤖 Chatbot: I have to call you James.
You: My name Syam
🤖 Chatbot: (laughs) Yes.
You: Do you speak English?
🤖 Chatbot: 
You: Are you  Machine or a Man?
🤖 Chatbot: I don't mind Machine. I'm a man, and there are plenty of men out here
You: Have you been to US before?
🤖 Chatbot: The same country
You: What did you eat in breakfast?
🤖 Chatbot: Bread
You: exit
🤖 Chatbot: Goodbye!


V2

In [11]:
from transformers import pipeline, set_seed
import re

# Load GPT-2 generator pipeline
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

chat_history = ""

print("🤖 Chatbot: Hello! I’m a simple chatbot. Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("🤖 Chatbot: Goodbye!")
        break

    # Append user input to history
    chat_history += f"You: {user_input}\nChatbot:"

    # Generate response using GPT-2 with truncation fix
    response = generator(
        chat_history,
        max_new_tokens=50,            # ✅ use max_new_tokens instead of max_length
        num_return_sequences=1,
        pad_token_id=50256,
        truncation=True               # ✅ optional: allows truncation of input if it's too long
    )

    # Extract the chatbot's reply
    full_reply = response[0]["generated_text"]
    bot_reply = full_reply[len(chat_history):].split("\n")[0]

    # Clean bot's reply if needed
    bot_reply = re.sub(r"<[^>]+>", "", bot_reply).strip()

    print("🤖 Chatbot:", bot_reply)

    # Update history
    chat_history += f" {bot_reply}\n"


Device set to use cpu


🤖 Chatbot: Hello! I’m a simple chatbot. Type 'exit' to stop.

You: Hi. What is your name?
🤖 Chatbot: my name is Dan.
You: Hi Dan! How are you doing today?
🤖 Chatbot: ok, that is nice. thank you, good to see you in a moment
You: Where are you from?
🤖 Chatbot: ok, what do you want to ask?
You: Can you tell me joke?
🤖 Chatbot: okay, maybe let me catch some people who are from another dimension.
You: Are you a man or a machine?
🤖 Chatbot: yeah, I know the word.
You: Do you speak English?
🤖 Chatbot: you do. i cant speak so i give you the address.
You: bye
🤖 Chatbot: ok
You: exit
🤖 Chatbot: Goodbye!


 DialoGPT and GPT-Neo

```



In [ ]:
# Install transformers if not already
!pip install transformers --quiet

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load GPT-Neo
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Store conversation
chat_history = ""

print("🤖 Chatbot: Hello! I'm your AI companion. Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("🤖 Chatbot: Goodbye!")
        break

    # Build prompt with history
    chat_history += f"Human: {user_input}\nAI:"

    # Tokenize prompt and generate response
    inputs = tokenizer(chat_history, return_tensors="pt")
    attention_mask = inputs["attention_mask"]  # 👈 Fixes the warning
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=attention_mask,
        max_new_tokens=50,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        pad_token_id=tokenizer.eos_token_id  # To avoid padding errors
    )

    # Decode only new generated part
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    reply = generated_text[len(chat_history):].split("Human:")[0].strip()

    print(f"🤖 Chatbot: {reply}")
    # Update history with bot reply
    chat_history += f" {reply}\n"


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

🤖 Chatbot: Hello! I'm your AI companion. Type 'exit' to stop.

You: Hi, do have a name?
🤖 Chatbot: *t*
You: is that what your name is?
🤖 Chatbot: *t*
You: will you answer my questions?
🤖 Chatbot: *t*
You: Exit
🤖 Chatbot: Goodbye!


DialoGPT

In [14]:
# Install required packages
!pip install transformers --quiet

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load pretrained DialoGPT (medium version is a good balance)
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Chat loop
print("🤖 Chatbot: Hello! I'm your AI companion. Type 'exit' to stop.\n")
chat_history_ids = None

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("🤖 Chatbot: Goodbye!")
        break

    # Encode the new user input
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append the new user input to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids

    # Generate a response
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.75
    )

    # Decode the response and print
    bot_reply = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"🤖 Chatbot: {bot_reply}")


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

🤖 Chatbot: Hello! I'm your AI companion. Type 'exit' to stop.

You: Hi, what is your name?
🤖 Chatbot: He's not a rapper.
You: Do you have a name?
🤖 Chatbot: I don't know, what's yours?
You: My name is Syam Sagar. what is your's?
🤖 Chatbot: I don't know, what's yours?
You: do u have a name?
🤖 Chatbot: I don't know, what is yours?
You: are u a chatbot?
🤖 Chatbot: Yes, yes, I am
You: what do you do?
🤖 Chatbot: I'm a computer.
You: what are your skills?
🤖 Chatbot: I am computer
You: what do u do?
🤖 Chatbot: I don't know, what do you do?
You: I am an IT professional
🤖 Chatbot: 
You: do u have knowledge base?
🤖 Chatbot: 
You: will u answer my questions?
🤖 Chatbot: 
You: whwre are u from ?
🤖 Chatbot: 
You: do u answer questions?
🤖 Chatbot: 
You: can u complete this story.. once upon a time there is a king
🤖 Chatbot: 
You: exit
🤖 Chatbot: Goodbye!
